In [1]:
import pandas as pd
from neuralprophet import NeuralProphet, set_log_level
import pickle

set_log_level("ERROR")


In [2]:

target_column = 'Total Inflow hrly'

df = pd.read_csv('https://raw.githubusercontent.com/drdevinhopkins/hourly-report/main/data/since-2020.csv')
df.ds = pd.to_datetime(df.ds)
df = df.drop(['Date', 'Time'], axis=1)
df = df.sort_values(by='ds', ascending=True)
df.rename(columns = {target_column:'y', 'Adm. requests cum':"Adm requests cum", 'Pts.waiting for admission CUM':'Pts waiting for admission CUM'}, inplace = True)
df.dropna(inplace=True)
regressors = df.columns.tolist()
regressors.remove('y')
regressors.remove('ds')

df

,Stretcher Pts hrly,Stretcher Pts cum,Ambulatory Pts hrly,Ambulatory Pts cum,y,Total Inflow cum,Ambulances hrly,Ambulances cum,FLS hrly,Adm requests cum,...,GARAGE patient TBS,Consults > 2h in Vertical Except IM,Consult for IM >4h in Vertical,Plain films reqs > 2 hr in Vertical,CTs reqs > 2 hrs in Vertical,Psych Stretcher Pts1pt,Psych pts waiting for admission,ds,Total Pod TBS,Total Vertical TBS
7659,9,108,5,142,14,250,6,39,0,27,...,1.0,1,0,0,3,0,0,2021-07-26 19:00:00,10,18.0
7658,5,113,6,148,11,261,2,41,0,33,...,1.0,1,0,0,3,1,0,2021-07-26 20:00:00,12,9.0
7657,5,118,2,150,7,268,1,42,0,34,...,0.0,1,0,0,4,4,0,2021-07-26 21:00:00,8,4.0
7656,5,123,8,158,13,281,2,44,0,34,...,0.0,1,0,0,3,5,0,2021-07-26 22:00:00,9,6.0
7655,3,126,7,165,10,291,3,47,0,37,...,1.0,0,0,1,3,6,0,2021-07-26 23:00:00,10,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,9,34,11,54,20,88,1,11,0,10,...,0.0,3,0,0,2,15,11,2022-06-12 11:00:00,3,21.0
3,7,41,6,60,13,101,1,12,1,12,...,0.0,4,0,0,1,15,11,2022-06-12 12:00:00,3,22.0
2,13,54,2,62,15,116,2,14,0,16,...,0.0,5,0,0,1,14,12,2022-06-12 13:00:00,1,29.0
1,11,65,6,68,17,133,2,16,0,15,...,0.0,4,0,0,1,14,12,2022-06-12 14:00:00,2,30.0


In [13]:
weather = pd.read_csv('https://raw.githubusercontent.com/drdevinhopkins/hourly-report/main/data/weatherArchiveAndForecast.csv')
weather.ds = pd.to_datetime(weather.ds)
weather.ds

0       2020-01-01 00:00:00
1       2020-01-01 01:00:00
2       2020-01-01 02:00:00
3       2020-01-01 03:00:00
4       2020-01-01 04:00:00
                ...        
21453   2022-06-13 19:00:00
21454   2022-06-13 20:00:00
21455   2022-06-13 21:00:00
21456   2022-06-13 22:00:00
21457   2022-06-13 23:00:00
Name: ds, Length: 21458, dtype: datetime64[ns]

In [17]:
df = df.merge(weather, on='ds')


In [19]:
df.tail()

,Stretcher Pts hrly,Stretcher Pts cum,Ambulatory Pts hrly,Ambulatory Pts cum,y,Total Inflow cum,Ambulances hrly,Ambulances cum,FLS hrly,Adm requests cum,...,Psych pts waiting for admission,ds,Total Pod TBS,Total Vertical TBS,temp,dew,humidity,precip,windspeed,sealevelpressure
7642,9,34,11,54,20,88,1,11,0,10,...,11,2022-06-12 11:00:00,3,21.0,22.4,13.3,56.38,0.0,5.4,1012.1
7643,7,41,6,60,13,101,1,12,1,12,...,11,2022-06-12 12:00:00,3,22.0,23.8,12.0,47.31,0.0,7.6,1011.6
7644,13,54,2,62,15,116,2,14,0,16,...,12,2022-06-12 13:00:00,1,29.0,19.4,14.7,74.62,0.3,5.5,1011.8
7645,11,65,6,68,17,133,2,16,0,15,...,12,2022-06-12 14:00:00,2,30.0,20.9,17.1,79.21,0.0,3.7,1010.7
7646,7,72,12,80,19,152,1,17,1,17,...,11,2022-06-12 15:00:00,2,31.0,22.7,16.3,67.17,1.2,9.7,1010.0


In [20]:
weather_regressors = ['temp','dew','humidity','precip','windspeed','sealevelpressure']

In [22]:

m = NeuralProphet(
  # growth='off',
  yearly_seasonality=False,
  weekly_seasonality=True,
  daily_seasonality=True,
  n_lags=48,
  n_forecasts=12,
  changepoints_range=0.95,
  n_changepoints=50,
  num_hidden_layers=4,
  d_hidden=36,
  learning_rate=0.005,
)
for reg in weather_regressors:
  m = m.add_future_regressor(name=reg)
m = m.add_lagged_regressor(names=regressors)
m = m.add_country_holidays("CA")
metrics = m.fit(df, 
                freq='H', 
                # progress='plot'
                )
print(metrics.tail(1))

# with open('models/inflow_forecast_model.pkl', "wb") as f:
#     pickle.dump(m, f)

WARNING - (py.warnings._showwarnmsg) - /opt/python/3.10.4/lib/python3.10/site-packages/neuralprophet/utils.py:251: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  additive_events_dims = additive_events_dims.append(

WARNING - (py.warnings._showwarnmsg) - /opt/python/3.10.4/lib/python3.10/site-packages/neuralprophet/utils.py:251: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  additive_events_dims = additive_events_dims.append(

WARNING - (py.warnings._showwarnmsg) - /opt/python/3.10.4/lib/python3.10/site-packages/neuralprophet/utils.py:251: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  additive_events_dims = additive_events_dims.append(

WARNING - (py.warnings._showwarnmsg) - /opt/python/3.10.4/lib/python3.10/site-packages/neur

     SmoothL1Loss       MAE      RMSE  RegLoss
109      0.001885  0.980435  1.286023      0.0


In [24]:
future = m.make_future_dataframe(df, periods=12) # periods=m.n_forecasts, n_historic_predictions=False


ValueError: Future values of all user specified regressors not provided

In [ ]:


future = loaded_model.make_future_dataframe(df, periods=12) # periods=m.n_forecasts, n_historic_predictions=False

forecast = loaded_model.predict(future, raw=True, decompose=False)

start = forecast.values.tolist()[0][0]
forecast_length = len(forecast.values.tolist()[0][1:])

forecast_output = pd.DataFrame(columns=['ds','inflow','timestamp'])
forecast_output['ds'] = pd.date_range(start=start, periods=forecast_length, freq='H')
forecast_output['inflow'] = forecast.values.tolist()[0][1:]
forecast_output['timestamp'] = pd.Timestamp.now().round('S').replace(tzinfo=None)
forecast_output.to_csv('forecasts/inflow.csv', index=False)